In [48]:
import os
import numpy as np
import glob
import shutil
import xarray as xr
from astropy.time import Time
import museval
from museval.utils import get_response, find_response, aia_synthesis, wavelength_in_cube
from museval.io import create_session, is_complete
from muse.synthesis.synthesis import transform_resp_units, vdem_synthesis
from matplotlib import colors
import matplotlib.pyplot as plt
from muse.instr.utils import convert_resp2muse_ciresp
import astropy.units as u
from matplotlib.ticker import PercentFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.wcs import WCS
from astropy.time import Time
import sunpy.map
import sunpy.visualization.colormaps as cm
from sunpy.time import parse_time
from dateutil.tz import gettz
import astropy.io.ascii as ascii
from astropy.coordinates import SkyCoord
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=UserWarning, append=True)
warnings.filterwarnings(
    "ignore",
    message="Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata.*",
    category=RuntimeWarning
) #To ignore this  annoying Zarr warning

In [62]:
os.environ['RESPONSE'] = '/Users/souvikb/MUSE_outputs/response/'
os.environ['text_files_path'] = '/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/'
os.environ['eis_data_path'] = '/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/'
os.environ['vdem_path'] = '/Users/souvikb/MUSE_outputs/vdems/'
work_dir = '/Users/souvikb/MUSE_outputs/pipeline_figs_sims/'
##---- interactively select a VDEM based on your choice. A new dialogue box opens. -----##
import tkinter as tk
from tkinter import filedialog
root = tk.Tk()
root.withdraw()  # Hide the root window
vdem_file = filedialog.askdirectory(
    title="Select a VDEM .zarr file",
    initialdir=os.environ['vdem_path']
)
print("Selected folder:", vdem_file)

Selected folder: /Users/souvikb/MUSE_outputs/vdems/vdems_bifrost_164_hion/vdem_164.zarr


In [63]:
txt_files = glob.glob(os.path.join(os.environ['text_files_path'], "*.txt"))
txt_files = sorted(txt_files)
# txt_files
MHD_code, snapname = [s.strip() for s in input("Enter the MHD code and snapname, separated by a comma: ").split(",")]

In [65]:
obs_dates = ascii.read(txt_files[0])
obs_dates.add_index("date_begin_EIS")

for date in tqdm(obs_dates['date_begin_EIS']):
    # print(date)
    date_sensitive = True
    unit = 'DN'
    date = Time(date,format='isot',scale='utc') # astropy time object
    print(f'Date Sensitive AIA {unit} Response from {date.strftime("%b %y")}')
    aia_resp = get_response(date = date, channels=[131,171,193,211],save_response=True, units=unit)
    muse_AIA = aia_synthesis(aia_resp, work_dir, vdem_file, swap_dims = False)
    print('Creating historgrams from the synthesized AIA data')
    fig,ax = plt.subplots(ncols=1,nrows=2,figsize=(4,8),sharex=True)
    ax.ravel()
    colors_channel = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    channels = [131,171,193,211]
    num_bins=40
    for i, channel in enumerate([131,171,193,211]):
        synth_aiadata = muse_AIA.flux.sel(channel=channel).to_numpy()
        # bins = np.linspace(muse_AIA.flux.sel(channel=channel).min(), muse_AIA.flux.sel(channel=channel).max(), num_bins)
        min_val = max(muse_AIA.flux.sel(channel=channel).min(), 1e-1)  # avoid log(0)
        max_val = muse_AIA.flux.sel(channel=channel).max()
        bins = np.logspace(np.log10(min_val), np.log10(max_val), num_bins)
        hist, bin_edges = np.histogram(synth_aiadata, bins=bins)
        
        ax[0].hist(synth_aiadata.ravel(), bins=bins, histtype='step',color=colors_channel[i], 
                   label=f'AIA {channel}',weights=np.ones(len(synth_aiadata.ravel())) / len(synth_aiadata.ravel()))
        
        ax[0].legend(loc = 'best')
        ax[0].set_xscale('log')
        ax[0].set_ylabel('Fraction')

        ax[1].hist(synth_aiadata.ravel(), bins=bins, histtype='step', color=colors_channel[i],cumulative=True,
                weights=np.ones(len(synth_aiadata.ravel())) / len(synth_aiadata.ravel()))
        ax[1].set_xlabel(r'Intensity [DNs$^{-1}$]')
        ax[1].set_xscale('log')
        ax[1].set_ylabel('ECDF')
        ax[1].axvline(x=np.mean(synth_aiadata.ravel()),color=colors_channel[i],lw=0.5,ls='--',
                      label=f'$\mu_{{{channel}}}$ {np.mean(synth_aiadata.ravel()):.2f}')
        ax[1].set_xlim([1,muse_AIA.flux.max().to_numpy()*1.1])
        ax[1].legend(loc='best', fontsize='small')
    fig.suptitle(f'{MHD_code} simulation: {snapname}',y=0.98,horizontalalignment='center')
    plt.tight_layout()
    # plt.show()
    os.makedirs(f'{work_dir}/{MHD_code}_{snapname}_synth_histograms/', exist_ok=True)
    fig.savefig(f'{work_dir}/{MHD_code}_{snapname}_synth_histograms/histograms.png', dpi=300,bbox_inches='tight')
    plt.close(fig)

    date_begin_EIS = obs_dates.loc[date]["date_begin_EIS"]
    cutouts_data_path = os.path.join(os.environ['eis_data_path'], "SDO_EIS_cutouts_"+date_begin_EIS)
    cutout_files = sorted(glob.glob(os.path.join(cutouts_data_path, 'cutout_*.fits')))
    print('The cutout files are:\n' + '\n'.join(cutout_files))
    if not cutout_files:
        print(f"No cutout files found in {cutouts_data_path}")
        continue
    AIA_DATA = {} # Initialize the dictionary
    # Dynamically calculate figure size based on the number of subplots and aspect ratio
    for idx_line, line in enumerate(channels):
        AIA_DATA[line] = sunpy.map.Map(cutout_files[idx_line])
    # Number of panels
    n_lines = len(channels)
    # Get one map to measure aspect ratio
    sample_map = AIA_DATA[channels[0]]
    ny, nx = sample_map.data.shape
    aspect = ny / nx  # height / width

    # Set figure size dynamically
    panel_width = 4  # good width per panel (in inches)
    fig_width = panel_width * n_lines
    fig_height = panel_width * aspect
    fig = plt.figure(figsize=(fig_width, fig_height))
    
    for idx_line, line in enumerate(channels):
        AIA_DATA[line] = sunpy.map.Map(cutout_files[idx_line])
        exp_time = AIA_DATA[line].exposure_time
        data = AIA_DATA[line]/exp_time
        ax = fig.add_subplot(1, n_lines, idx_line + 1, projection=AIA_DATA[line].wcs)
        ##Plotting the AIA cutouts
        im = data.plot(axes=ax, vmin=0)
        # ax.set_title(data.wavelength.to_string('latex'), fontsize=10)
        ax.grid(False)
        lon = ax.coords[0]
        lat = ax.coords[1]
        lon.set_ticks_position('b')      # ticks on bottom
        lon.set_ticklabel_position('b')  # bottom

        lat.set_ticks_position('l')      # ticks on left
        lat.set_ticklabel_position('l')  # left
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.05, axes_class=plt.Axes)
        cbar = fig.colorbar(im, cax=cax)
        cbar.ax.tick_params(direction='out')
        cbar.ax.yaxis.set_ticks_position('right')
        cbar.ax.yaxis.set_label_position('right')
        cbar.ax.yaxis.tick_right()
        cbar.set_label(r'DNs$^{-1}$')
    plt.tight_layout()
    # plt.show()
    os.chdir(cutouts_data_path)
    output_dir = './pipeline_figs'
    os.makedirs(output_dir, exist_ok=True) 
    fig.savefig(os.path.join(output_dir,f"{date}_cutouts.png"),dpi=300,bbox_inches='tight')
    plt.close(fig)

    ## ------ Comparison between given AIA lines and synthesized AIA data ------ ##
    region = 'plage'
    fig, ax = plt.subplots(2,2,figsize=(10,10))
    for i,channel in enumerate(channels):
        exp_time = AIA_DATA[channel].exposure_time
        data = np.ravel(AIA_DATA[channel].data/exp_time)
        bf = muse_AIA.flux.sel(channel=channel).to_numpy()
        combined = np.concatenate([data.value, bf.ravel()])
        min_val = max(combined.min(), 1e-1)  # avoiding log(0). This can be nasty otherwise
        max_val = combined.max()
        bins_new = np.logspace(np.log10(min_val), np.log10(max_val), num_bins)
        # bins_new = np.linspace(combined.min(), combined.max(), 60)
        hist, bin_edges = np.histogram(data.value, bins=bins_new)
        ax[i//2][i%2].hist(data.value, bins=bins_new, label=f'AIA {region}',cumulative=True, histtype='step',
                           weights=np.ones(len(data.value)) / len(data.value), color=colors_channel[i])
        ax[i//2][i%2].axvline(x=np.mean(data.value),color=colors_channel[i],lw=1,ls='-',
                              label=f'$\mu_{{\mathrm{{obs}}}}$: {np.mean(data.value):.2f} [DNs$^{-1}$]')

        ax[i//2][i%2].hist(bf.ravel(), bins=bins_new, label=f'{MHD_code} {snapname}',cumulative=True, histtype='step',
                           weights=np.ones(len(bf.ravel())) / len(bf.ravel()),color='tab:purple',ls='-.',lw=2.)
        ax[i//2][i%2].axvline(x=np.mean(bf.ravel()),color='tab:purple',lw=1,ls='-.',label=f'$\mu_{{\mathrm{{syn}}}}$: {np.mean(bf.ravel()):.2f} [DNs$^{-1}$]')
        ax[i//2][i%2].set_xlabel(fr'{channel} Intensity [DNs$^{-1}$]')
        ax[i//2][i%2].set_xlim([1, max_val * 1.1])
        # if channel == 171:
        #     ax[i//2][i%2].set_xlim([1, 1e4])
        # elif channel == 193:
        #     ax[i//2][i%2].set_xlim([1, 1e4])
        # elif channel == 211:
        #     ax[i//2][i%2].set_xlim([1, 3e3])
        # elif channel == 131:
        #     ax[i//2][i%2].set_xlim([1, 1e3])
        ax[i//2][i%2].set_xscale('log')
        ax[i//2][i%2].set_ylabel('ECDF')
        ax[i//2][i%2].legend(loc='lower right')
        ax[i//2][i%2].axhline(y=0.25,color='black',ls='--')
        ax[i//2][i%2].axhline(y=0.75,color='black',ls='--')

    date = AIA_DATA[channel].date.strftime('%Y-%m-%d')
    title = f"AIA {region} {date} vs {MHD_code} {snapname}"
    fig.suptitle(title,y=0.98)
    fig.suptitle(title,y=0.98)
    plt.tight_layout()
    # plt.show()

    os.chdir(cutouts_data_path)
    output_dir = './pipeline_figs'
    os.makedirs(output_dir, exist_ok=True)
    fig.savefig(os.path.join(output_dir,f"{date}_{MHD_code}_{snapname}_comparison_hist.png"),dpi=300,bbox_inches='tight')
    plt.close(fig)

  0%|          | 0/12 [00:00<?, ?it/s]

Date Sensitive AIA DN Response from Jun 17
*** Nearest response function is 12 months from obs_date 
*** /Users/souvikb/MUSE_outputs/response/aia_resp_DN_Jun18.zarr already exists! Reading...
The channels of the response function match.
*** Work directory is /Users/souvikb/MUSE_outputs/pipeline_figs_sims/
*** Loading /Users/souvikb/MUSE_outputs/vdems/vdems_bifrost_164_hion/vdem_164.zarr into vdem
Creating historgrams from the synthesized AIA data
The cutout files are:
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-06-07T04:41:20/cutout_AIA_131.fits
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-06-07T04:41:20/cutout_AIA_171.fits
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-06-07T04:41:20/cutout_AIA_193.fits
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-06-07T04:41:20/cutout_AIA_211.fits
/Users/souvikb/MUSE_outputs/EIS_IRI

  8%|▊         | 1/12 [00:06<01:14,  6.74s/it]

Date Sensitive AIA DN Response from Jun 17
*** Nearest response function is 12 months from obs_date 
*** /Users/souvikb/MUSE_outputs/response/aia_resp_DN_Jun18.zarr already exists! Reading...
The channels of the response function match.
*** Work directory is /Users/souvikb/MUSE_outputs/pipeline_figs_sims/
*** Loading /Users/souvikb/MUSE_outputs/vdems/vdems_bifrost_164_hion/vdem_164.zarr into vdem
Creating historgrams from the synthesized AIA data
The cutout files are:
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-06-08T01:02:39/cutout_AIA_131.fits
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-06-08T01:02:39/cutout_AIA_171.fits
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-06-08T01:02:39/cutout_AIA_193.fits
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-06-08T01:02:39/cutout_AIA_211.fits
/Users/souvikb/MUSE_outputs/EIS_IRI

 17%|█▋        | 2/12 [00:13<01:06,  6.66s/it]

...

 83%|████████▎ | 10/12 [00:55<00:10,  5.21s/it]

Date Sensitive AIA DN Response from Nov 17
*** Nearest response function is 7 months from obs_date 
*** /Users/souvikb/MUSE_outputs/response/aia_resp_DN_Jun18.zarr already exists! Reading...
The channels of the response function match.
*** Work directory is /Users/souvikb/MUSE_outputs/pipeline_figs_sims/
*** Loading /Users/souvikb/MUSE_outputs/vdems/vdems_bifrost_164_hion/vdem_164.zarr into vdem
Creating historgrams from the synthesized AIA data
The cutout files are:
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-11-08T08:26:53/cutout_AIA_131.fits
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-11-08T08:26:53/cutout_AIA_171.fits
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-11-08T08:26:53/cutout_AIA_193.fits
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-11-08T08:26:53/cutout_AIA_211.fits
/Users/souvikb/MUSE_outputs/EIS_IRIS

 92%|█████████▏| 11/12 [01:02<00:05,  5.67s/it]

Date Sensitive AIA DN Response from Nov 17
*** Nearest response function is 7 months from obs_date 
*** /Users/souvikb/MUSE_outputs/response/aia_resp_DN_Jun18.zarr already exists! Reading...
The channels of the response function match.
*** Work directory is /Users/souvikb/MUSE_outputs/pipeline_figs_sims/
*** Loading /Users/souvikb/MUSE_outputs/vdems/vdems_bifrost_164_hion/vdem_164.zarr into vdem
Creating historgrams from the synthesized AIA data
The cutout files are:
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-11-08T23:04:13/cutout_AIA_131.fits
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-11-08T23:04:13/cutout_AIA_171.fits
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-11-08T23:04:13/cutout_AIA_193.fits
/Users/souvikb/MUSE_outputs/EIS_IRIS_QS_obs/Plage_datasets/HOP_307/SDO_EIS_cutouts_2017-11-08T23:04:13/cutout_AIA_211.fits
/Users/souvikb/MUSE_outputs/EIS_IRIS

100%|██████████| 12/12 [01:09<00:00,  5.78s/it]
